<a href="https://colab.research.google.com/github/HyunLee103/AI_Hackathon_konkuk/blob/master/word2vec-gyu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import joblib
import os

# token: list
# label: pd.Series

base_dir = 'data'

train_token = joblib.load(os.path.join(base_dir, 'train_tokens.pickle'))
train_label = joblib.load(os.path.join(base_dir, 'train_label.pickle'))

test_token = joblib.load(os.path.join(base_dir, 'test.pickle'))
test_label = joblib.load(os.path.join(base_dir, 'test_label.pickle'))

tfidf_vocab = joblib.load(os.path.join('data', 'top_10000.pickle'))

print(train_token[:3])
print(test_token[:3])

print(tfidf_vocab[:10])
print(len(tfidf_vocab))

[['더빙', '진짜', '짜증나다', '목소리'], ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '가볍다'], ['무재', '밓었', '다그', '래서', '추천']]
[['굳', 'ㅋ'], ['GDNTOPCLASSINTHECLUB'], ['뭐', '이', '평점', '들', '나쁘진', '않지만', '점', '짜', '리', '더', '더욱', '아니잖아']]
['영화', '없다', '있다', '좋다', '너무', '재밌다', '정말', '같다', '진짜', '아니다']
10000


In [ ]:
from gensim.models import Word2Vec

embedding_dim = 128

word2vec_model = Word2Vec([tfidf_vocab], size=embedding_dim, min_count=1, sg=1)
word2vec_model.save('model/word2vec_model')

In [ ]:
import numpy as np

def build_matrix(tokens_list, word_table, embedding_dim):
    size = len(tokens_list)
    matrix = np.zeros((size, embedding_dim))
    for i, tokens in enumerate(tokens_list):
        vectorized_tokens = np.array([
            word_table[token] if token in word_table else np.zeros(embedding_dim)
            for token in tokens 
        ])
        matrix[i] = vectorized_tokens.mean(0)
    return np.nan_to_num(matrix)

train_x = build_matrix(train_token, word2vec_model, embedding_dim)
test_x = build_matrix(test_token, word2vec_model, embedding_dim)

In [ ]:
# 진짜 간단하게 만들어보기
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KDTree, KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

cfs = [
    GaussianNB(),
    DecisionTreeClassifier(criterion='entropy'),
    # KNeighborsClassifier(n_neighbors=2)
]

for cf in cfs:
    cf.fit(train_x, train_label)

In [ ]:
from sklearn.metrics import accuracy_score

for cf in cfs:
    predictions = cf.predict(test_x)
    acc = accuracy_score(test_label, predictions)
    print(cf, acc)

GaussianNB() 0.5174441076550644
DecisionTreeClassifier(criterion='entropy') 0.5574790975852879
